# LinkedIn Post Generator


---
## 1.&nbsp; Installations and Settings 🛠️

**faiss-cpu** is a library that provides a fast and efficient database for storing and retrieving our numerical summaries of information.

In [37]:
%%bash
pip install -qqq -U langchain-huggingface
pip install -qqq -U langchain
pip install -qqq -U langchain-community
pip install -qqq -U faiss-cpu
pip install -qU langchain_community CSVLoader
pip install -r requirements.txt
pip install -U streamlit
pip install -qU pypdf
python -m spacy download en_core_web_sm

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: Could not find a version that satisfies the requirement base64 (from versions: none)
ERROR: No matching distribution found for base64


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 7.2 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


time: 26 s (started: 2024-11-04 13:11:18 +01:00)


In [40]:
from Keys__tokens import *
HUGGINGFACEHUB_API_TOKEN = "HugFaceWBS"

time: 1.16 ms (started: 2024-11-04 13:11:54 +01:00)


In [42]:
import os
from Keys__tokens import *
#from google.colab import userdata # we stored our access token as a colab secret

os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

time: 200 μs (started: 2024-11-04 13:11:56 +01:00)


In [44]:
from collections import Counter
import re
import numpy as np
from Keys__tokens import *

time: 164 μs (started: 2024-11-04 13:11:58 +01:00)


---
## 2.&nbsp; Merged Code 🧠

In [46]:
#%%writefile streamlit_app.py

from langchain_huggingface import HuggingFaceEndpoint, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import CSVLoader
import streamlit as st
import os
from Keys__tokens import *
from collections import Counter
import re


#Importing keys
HUGGINGFACEHUB_API_TOKEN = "HugFaceWBS"

#from google.colab import userdata # we stored our access token as a colab secret
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN



#llm selection
from langchain_huggingface import HuggingFaceEndpoint

# This info's at the top of each HuggingFace model page
hf_model = "mistralai/Mixtral-8x7B-Instruct-v0.1"

# Create the HuggingFaceEndpoint and add the `add_to_git_credential=True` option
llm = HuggingFaceEndpoint(
    repo_id=hf_model,
    max_new_tokens=512,
    huggingfacehub_api_token = HugFaceWBS
    #add_to_git_credential=True  # This saves the token to git credentials
)


#Retrieval Augmented Generation
csv_01 = ("Database.csv")

linkedin_database = [csv_01]



#Loading the data
from langchain_community.document_loaders.csv_loader import CSVLoader

pages = []

for sheet in linkedin_database:
    loader = CSVLoader(sheet)
    pages.extend(loader.load())



#Splitting the document
#from langchain.text_splitter import RecursiveCharacterTextSplitter

#text_splitter = RecursiveCharacterTextSplitter(chunk_size=800,
#                                              chunk_overlap=150)

#docs = text_splitter.split_documents(pages)




#Creating vectors with embeddings
from langchain_huggingface import HuggingFaceEmbeddings


# embeddings
embedding_model = "sentence-transformers/all-MiniLM-l6-v2"
embeddings_folder = "Content"

embeddings_instance = HuggingFaceEmbeddings(model_name=embedding_model, cache_folder=embeddings_folder)

# Generate embeddings for documents
embeddings = embeddings_instance.embed_documents([doc.page_content for doc in pages])

# Convert embeddings to numpy array of float32
embedding_vectors = np.array(embeddings, dtype=np.float32)



#Creating a vector database
#from langchain.vectorstores import FAISS

vector_db = FAISS.from_documents(pages, embeddings_instance)

#Once the database is made, you can save it to use over and over again in the future.

vector_db.save_local("Content/faiss_index")
#Here's the code to load it again.
#vector_db = FAISS.load_local("/Users/alexandrealbieri/Documents/Personal/Berlin/WBS Coding School/WBS Bootcamp/Data Science/Generative AI/Content/faiss_index", embeddings, allow_dangerous_deserialization=True)


#Adding a prompt
from langchain.prompts.prompt import PromptTemplate

input_template = """You are tasked with generating engaging, professional LinkedIn post based on the database. Create an unique text each time based on different posts, and also different from the text in the memory.
Keep it brief and to the point (maximum 150 words). Don't mention other users. 

Context to answer question: {context}

Question to be answered: {question}
Response:"""

prompt = PromptTemplate(template=input_template,
                        input_variables=["context", "question"])


#RAG - Chaining it all together
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = vector_db.as_retriever(search_kwargs={"k": 4}), # top 2 results only, speed things up
    return_source_documents = False,
    chain_type_kwargs = {"prompt": prompt},
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/alexandrealbieri/.cache/huggingface/token
Login successful
time: 17.1 s (started: 2024-11-04 13:12:02 +01:00)
